In [1]:
#import os
#os.environ['TRANSFORMERS_CACHE'] = '/home/smoeding/caches/'
#os.environ['XDG_CACHE_HOME'] = '/home/smoeding/caches/'

In [2]:
import math

import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import torch
import pandas as pd
import math
from sklearn.metrics import accuracy_score

In [3]:
from transformers import BitsAndBytesConfig

#model_name = "fully_trained_model/"
#model_name = "trained_on_chunk/"
model_name = "medalpaca/medalpaca-7b"
#model_name = "LeoLM/leo-mistral-hessianai-7b"
#model_name = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = False
tokenizer.add_bos_token, tokenizer.add_eos_token

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


(True, False)

In [5]:
generation_config = GenerationConfig(max_new_tokens=50,
                                    temperature=0.4,
                                    top_p=0.95,
                                    top_k=40,
                                    repetition_penalty=1.3,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    do_sample=True,
                                    use_cache=True,
                                    output_attentions=False,
                                    output_hidden_states=False,
                                    output_scores=False,
                                    remove_invalid_values=True
                                    )

In [6]:
prompt_case = "Es folgt eine Fallbeschreibung, dessen Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Beantworte die Frage, indem du die korrekte Antwortmöglichkeit wiedergibst. Die richtige Antwort ist: BEGINANTWORT"
prompt_few_shots = """Es folgt eine Fallbeschreibung, dessen Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Die Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL Ein 3 Tage altes männliches Neugeborenes wird mit zunehmender Somnolenz, Apathie, Erbrechen, erhöhter Atemfrequenz und epileptischem Anfall auf die pädiatrische Intensivstation aufgenommen. ENDFALL BEGINFRAGE Welcher der folgenden Laborparameter ist im Sinne einer raschen Diagnosefindung am wenigsten dringlich zu bestimmen? ENDFRAGE BEGINANTWORT 0) Laktat ENDANTWORT BEGINANTWORT 1) Glukose ENDANTWORT BEGINANTWORT 2) Ammoniak ENDANTWORT BEGINANTWORT 3) C-reaktives Protein ENDANTWORT BEGINANTWORT 4) Harnsäure ENDANTWORT. Beantworte die Frage, indem du die korrekte Antwortmöglichkeit wiedergibst. Die richtige Antwort ist: BEGINANTWORT 4) Harnsäure ENDANTWORT.
Es folgt eine Fallbeschreibung, dessen Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Die Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL Ein 19-jähriger Patient stellt sich bei Ihnen in der dermatologischen Sprechstunde vor, weil er seit einigen Jahren an einer Akne im Gesicht leide. Die Haut erscheine immer fettig und glänze häufig. Die Akne sei nun in den letzten 3-4 Monaten sehr viel schlimmer geworden und er wache morgens öfters auf einem blutigen Kopfkissen auf. Wenn neue Hauterscheinungen entstehen, dann sei dies gelegentlich auch sehr schmerzhaft.Am Vorabend habe er einen Pickel auf dem Nasenrücken, der ihn gestört habe, manuell entfernen wollen; jetzt sei es zu einer starken Schwellung im Augeninnenwinkel gekommen. Die ganze Region sei schmerzhaft.<br>Seine Mutter habe auch an Akne gelitten, der Vater nicht. Seine Schwester sei erst 12 Jahre alt, und bisher sei bei ihr keine Akne aufgetreten. Allergien habe er nicht. Weitere Grunderkrankungen sind nicht bekannt. Auf Nachfrage verneint der Patient eine Medikamenteneinnahme.Klinisch zeigen sich typische Akneeffloreszenzen, auch mit kleinen Knoten. Dazu findet sich an beiden Wangen und im Kinnbereich eine Vielzahl von zum Teil eingezogenen Narben. Im Bereich des Augeninnenwinkels links zeigt sich eine im Durchmesser ca. 2 cm große, hellrote Schwellung mit zentralem Eiter. Andere Körperregionen sind weniger betroffen, einzelne Hautveränderungen finden sich auch an beiden Schultern. Neurologische Störungen stellen sich nicht dar.<br>In den Laboruntersuchungen zeigt sich ein deutlich erhöhtes C-reaktives Protein (75 mg/L), das Blutbild ist unauffällig. Ein HIV-Test ist negativ. Eine Sinus-cavernosus-Thrombose kann bei dem jungen Mann ausgeschlossen werden. Sie informieren den Patienten über die Erkrankung, deren Ursachen und den Verlauf und besprechen das weitere therapeutische Vorgehen. Wegen des beginnenden Abszesses im inneren Augenwinkel veranlassen Sie zunächst eine orale Antibiotikatherapie mit Clindamycin für mehrere Tage. Hierunter kommt es innerhalb von 3 Tagen zu einer zügigen Rückbildung des Abszesses und der Schmerzen im Augenwinkel, das CRP ist nach 7 Tagen innerhalb des Referenzbereiches. Die Clindamycintherapie wird beendet. Anschließend empfehlen Sie dem Patienten eine Kombinationstherapie, die er für mehrere Wochen anwenden soll. Auch eine milde Reinigung der Haut mit pH-neutralen Waschsyndets wird empfohlen. Eine Kontrolluntersuchung in 8 Wochen wird vereinbart. ENDFALL BEGINFRAGE Welche der folgenden Kombinationen von Hautveränderungen liegt bei diesem Patienten am wahrscheinlichsten vor? ENDFRAGE BEGINANTWORT 0) Bläschen, Krusten ENDANTWORT BEGINANTWORT 1) Komedonen, Papeln ENDANTWORT BEGINANTWORT 2) Petechien, Schuppen ENDANTWORT BEGINANTWORT 3) Rhagaden, Quaddeln ENDANTWORT BEGINANTWORT 4) Quaddeln, Bläschen ENDANTWORT. Beantworte die Frage, indem du die korrekte Antwortmöglichkeit wiedergibst. Die richtige Antwort ist: BEGINANTWORT 1) Komedonen, Papeln ENDANTWORT.
"""

In [7]:
def prompt_model(prompt):
    input_tokens=tokenizer(prompt_few_shots + prompt, return_tensors="pt").to(model.device)
    output_tokens=model.generate(**input_tokens, generation_config=generation_config, pad_token_id=tokenizer.eos_token_id)[0]
    answer=tokenizer.decode(output_tokens, skip_special_tokens=True)
    return answer

In [8]:
questions_df = pd.read_csv("../output.csv")
questions_df["Prediction LLM"] = [-1 for i in range(len(questions_df))]
questions_df["Answer LLM"] = [-1 for i in range(len(questions_df))]

In [15]:
from tqdm import tqdm
import difflib

for index, row in tqdm(questions_df.iterrows()):
    if row["Answer LLM"] == -1:
        if type(row["Case"]) == str:
            prompt = prompt_case.replace("<INSERTFALL>",row["Case"])
        else:
            prompt = prompt_case.replace("<INSERTFALL>", "")
        prompt = prompt.replace("<INSERTFRAGE>",row["Question"])
        for i in range(5):
            prompt = prompt.replace("<INSERTANTWORT" + str(i) + ">",row["Answer "+ str(i+1)])
        answer = prompt_model(prompt).split("Die richtige Antwort ist: BEGINANTWORT")
        answer = answer[len(answer)-1].split("ENDANTWORT")[0]
        found = False
        for i in range(5):
            if row["Answer "+ str(i+1)].strip().lower().replace(".","").replace("\n","") == answer.strip().lower().replace(".","").replace("\n",""):
                found = i
                break
                
        #print(answer)
        
        
        if found != False:
            questions_df.loc[index, "Answer LLM"] = found
        questions_df.loc[index, "Prediction LLM"] = answer
print("Values set:", len(questions_df[questions_df["Answer LLM"] != -1]),"/",len(questions_df))


248it [02:38,  1.57it/s]

Values set: 124 / 248


In [16]:
questions_df.to_csv("medalpaca_output.csv")

In [11]:
y_pred = questions_df[questions_df["Answer LLM"] != -1]["Answer LLM"]
y_true = questions_df[questions_df["Answer LLM"] != -1]["Correct Answer"]

In [12]:
print("Accuracy:",accuracy_score(y_true, y_pred))

Accuracy: 0.14814814814814814
